In [9]:
import org.apache.spark._
import org.apache.spark.graphx._
// To make some of the examples work we will also need RDD
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import org.apache.log4j.Logger
import org.apache.log4j.Level
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf

import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import org.apache.log4j.Logger
import org.apache.log4j.Level
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf


In [16]:
sc.stop()

In [121]:
import scala.util.MurmurHash

import scala.util.MurmurHash


In [17]:
val spark = SparkSession.builder()
.appName("Dataframegiris")
.master("local[4]")
.config("spark.driver.memory","2g")
.config("spark.executor.memory","4g")
.getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@1b78a3d


In [155]:
val venues = spark.read.textFile("export_venues.csv")
val links = spark.read.textFile("export_links.csv")

venues: org.apache.spark.sql.Dataset[String] = [value: string]
links: org.apache.spark.sql.Dataset[String] = [value: string]


In [156]:
venues.show(3)

+--------------------+
|               value|
+--------------------+
|4c0e115bc700c9b6c...|
|4bed1e076e8c20a19...|
|4ce9209e0f196dcbd...|
+--------------------+
only showing top 3 rows



In [157]:
links.show(3)

+--------------------+
|               value|
+--------------------+
|4c0e115bc700c9b6c...|
|4c0e115bc700c9b6c...|
|4c0e115bc700c9b6c...|
+--------------------+
only showing top 3 rows



In [83]:
class Graph[VD, ED] (val vertices: VertexRDD[VD], val edges: EdgeRDD[ED])

defined class Graph
Companions must be defined together; you may wish to use :paste mode for this.


In [107]:
case class Venues(Name:String,lat:Float,lng:Float)


defined class Venues


In [147]:
def ayır(line:String):(Long,Venues)={
    val a = MurmurHash.stringHash(line.split(",")(0).toString)
    
    val b = line.split(",")(1).toString
    val c = line.split(",")(2).toFloat
    val d = line.split(",")(3).toFloat

    
    return (a, Venues(b,c,d))    
}


ayır: (line: String)(Long, Venues)


In [146]:
//val venuesRDD: RDD[(VertexId, Venues)] = venues.rdd.map(line => line.split(","))
//.map(line => (MurmurHash.stringHash(line(0).toString),(line(1).toString,line(2).toFloat,line(3).toFloat)) )

<console>: 64: error: type mismatch;

In [148]:
val venuesRDD: RDD[(VertexId, Venues)] =  venues.rdd.map(ayır)

venuesRDD: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, Venues)] = MapPartitionsRDD[45] at map at <console>:65


In [152]:
venuesRDD.take(3)

res22: Array[(org.apache.spark.graphx.VertexId, Venues)] = Array((914172510,Venues(Anıtkabir,39.926785,32.83659)), (-2086253975,Venues(Middle East Technical University (Orta Doğu Teknik Üniversitesi),39.90006,32.788715)), (919849682,Venues(HD İskender,39.95,32.830822)))


In [159]:
type Connection = Float

defined type alias Connection


In [165]:
val linksRDD: RDD[Edge[Connection]] = links.rdd.map{ line => 
  val vs = line.split(",");
  Edge(MurmurHash.stringHash(vs(0).toString), MurmurHash.stringHash(vs(1).toString), vs(2).toFloat)
}

linksRDD: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Connection]] = MapPartitionsRDD[57] at map at <console>:63


In [167]:
val venuesgraph = Graph(venuesRDD, linksRDD)

venuesgraph: org.apache.spark.graphx.Graph[Venues,Connection] = org.apache.spark.graphx.impl.GraphImpl@6769e3b1


In [175]:
sc.uiWebUrl

res32: Option[String] = Some(http://192.168.0.15:4040)


In [4]:
class Graph[VD, ED](
  val vertices: VertexRDD[VD],
  val edges: EdgeRDD[ED]) {
}

defined class Graph
Companions must be defined together; you may wish to use :paste mode for this.


In [7]:
class VertexProperty()
case class UserProperty(val name: String) extends VertexProperty
case class ProductProperty(val name: String, val price: Double) extends VertexProperty
// The graph might then have the type:

// Create an RDD for the vertices
val users: RDD[(VertexId, (String, String))] =
  sc.parallelize(Seq((3L, ("rxin", "student")), (7L, ("jgonzal", "postdoc")),
                       (5L, ("franklin", "prof")), (2L, ("istoica", "prof")),
                       (4L, ("peter", "student"))))
// Create an RDD for edges
val relationships: RDD[Edge[String]] =
  sc.parallelize(Seq(Edge(3L, 7L, "collab"),    Edge(5L, 3L, "advisor"),
                       Edge(2L, 5L, "colleague"), Edge(5L, 7L, "pi"),
                       Edge(4L, 0L, "student"),   Edge(5L, 0L, "colleague")))
// Define a default user in case there are relationship with missing user
val defaultUser = ("John Doe", "Missing")
// Build the initial Graph
val graph = Graph(users, relationships, defaultUser)
// Notice that there is a user 0 (for which we have no information) connected to users
// 4 (peter) and 5 (franklin).
graph.triplets.map(
  triplet => triplet.+ " is the " + triplet.attr + " of " + triplet.dstAttr._1
).collect.foreach(println(_))
// Remove missing vertices as well as the edges to connected to them
val validGraph = graph.subgraph(vpred = (id, attr) => attr._2 != "Missing")
// The valid subgraph will disconnect users 4 and 5 by removing user 0
validGraph.vertices.collect.foreach(println(_))
validGraph.triplets.map(
  triplet => triplet.srcAttr._1 + " is the " + triplet.attr + " of " + triplet.dstAttr._1
).collect.foreach(println(_))

rxin is the collab of jgonzal
franklin is the advisor of rxin
istoica is the colleague of franklin
franklin is the pi of jgonzal
peter is the student of John Doe
franklin is the colleague of John Doe
(2,(istoica,prof))
(3,(rxin,student))
(4,(peter,student))
(5,(franklin,prof))
(7,(jgonzal,postdoc))
rxin is the collab of jgonzal
franklin is the advisor of rxin
istoica is the colleague of franklin
franklin is the pi of jgonzal


defined class VertexProperty
defined class UserProperty
defined class ProductProperty
users: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, (String, String))] = ParallelCollectionRDD[0] at parallelize at <console>:28
relationships: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[String]] = ParallelCollectionRDD[1] at parallelize at <console>:33
defaultUser: (String, String) = (John Doe,Missing)
graph: org.apache.spark.graphx.Graph[(String, String),String] = org.apache.spark.graphx.impl.GraphImpl@796ee64a
validGraph: org.apache.spark.graphx.Graph[(String, String),String] = org.apache.spark.graphx.impl.GraphImpl@1af3d9ec
